In [1]:
import os
import yaml
import pandas as pd

# 列幅を広げる（制限をなくす場合は -1 を指定）
pd.options.display.max_colwidth = None

# 設定ファイルの読み込み
with open('config.yaml', 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

primary_folders = config['primary_folders']
secondary_folders = config['secondary_folders']
ignore_extensions = config['ignore_extensions']

In [2]:
# 患者候補フォルダのリストを格納するリスト
patient_candidates = []

# primary_folderとsecondary_folderの絶対パスを記録
primary_paths = {os.path.abspath(folder) for folder in primary_folders}
secondary_paths = {os.path.abspath(folder) for folder in secondary_folders}

# secondary_folder直下の患者候補フォルダをリストアップ
patient_paths = set()
for secondary_folder in secondary_folders:
    abs_secondary_folder = os.path.abspath(secondary_folder)
    if os.path.exists(abs_secondary_folder):
        for entry in os.listdir(abs_secondary_folder):
            entry_path = os.path.join(abs_secondary_folder, entry)
            if os.path.isdir(entry_path):
                patient_candidates.append({
                    'patient_folder_name': entry,
                    'patient_folder_path': os.path.abspath(entry_path),
                    'secondary_folder': abs_secondary_folder
                })

# 患者候補フォルダ一覧をDataFrameに変換して保存
df_patients = pd.DataFrame(patient_candidates)
df_patients.to_parquet('patients_candidates.parquet', index=False)
print("患者候補フォルダの一覧をpatients_candidates.parquetとして出力しました。")

患者候補フォルダの一覧をpatients_candidates.parquetとして出力しました。


In [22]:
from tqdm import tqdm
from pathlib import Path

# 例外ファイル・フォルダの格納リスト
exception_entries = []

# すべての対象となるファイル・フォルダの抽出
# Path オブジェクトを使って対象の絶対パスを取得
abs_paths = [
    str(path.resolve())
    for primary_folder in primary_folders
    for path in Path(primary_folder).rglob("*")
]

for abs_path in tqdm(abs_paths):
    # 患者フォルダそのもの、あるいは中身を除外
    if any(abs_path.startswith(patient_path) for patient_path in df_patients['patient_folder_path']):
        continue

    # secondaryフォルダそのものを除外
    if abs_path in {os.path.abspath(secondary_folder) for secondary_folder in secondary_folders}:
        continue

    # primaryフォルダそのものを除外
    if abs_path in {os.path.abspath(primary_folder) for primary_folder in primary_folders}:
        continue

    # 無視する拡張子のファイルを除外
    if any(abs_path.lower().endswith(ext.lower()) for ext in ignore_extensions):
        continue

    # リストに追加
    exception_entries.append({
        'type': 'folder' if os.path.isdir(abs_path) else 'file',
        'path': abs_path
    })

# 例外フォルダ・ファイル一覧をDataFrameに変換して保存
df_exceptions = pd.DataFrame(exception_entries)
df_exceptions.to_parquet('exceptions.parquet', index=False)
print("例外として検出されたファイル・フォルダをexceptions.parquetとして出力しました。")

df_exceptions

100%|██████████████████████████████████| 17013/17013 [00:00<00:00, 28095.12it/s]

例外として検出されたファイル・フォルダをexceptions.parquetとして出力しました。


,type,path
0,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/わ行/渡辺益務.ppt
1,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/わ行/Kｒ.渡辺綾子 初診.ppt
2,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/わ行/渡辺ミチ子 初診時.ppt
3,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/は行/藤田ヨシ子.ppt
4,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/は行/藤沢正義.ppt
5,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/は行/堀 巌.ppt
6,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/は行/笛木弘 初診時.ppt
7,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/は行/笛木弘 ２００８.2.5.ppt
8,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/は行/広澤照子.ppt
9,file,/Users/historoid/Documents/Res/DVI/preprocess/mydata/00_working/総合診療部/は行/本名きえこ.ppt


In [19]:
!pip install tqdm